In [34]:
# %load ../init.ipy
%reload_ext autoreload
%autoreload 2
from importlib import reload

import numpy as np
import holodeck as holo
import holodeck.single_sources as ss
from holodeck.constants import YR
import matplotlib.pyplot as plt
import scipy as sp

In [35]:
edges, number, fobs, exname = ss.example4(print_test=False)
hc_bg, hc_ss, ssidx, hsamp, bgpar, sspar = ss.ss_by_cdefs(edges, number, 30, params=True)
# example 4
dur = 5.0*YR/3.1557600
cad = .2*YR/3.1557600

12:26:38 INFO : zero_coalesced=True, zero_stalled=False [sam.py:dynamic_binary_number]
12:26:38 INFO : Stalled SAM bins based on GMT: 6.25e+03/1.56e+04 = 4.00e-01 [sam.py:static_binary_density]
12:26:38 INFO : Adding MMbulge scatter (3.4000e-01) [sam.py:static_binary_density]
12:26:38 INFO : 	dens bef: (0.00e+00, 5.88e-204, 7.46e-47, 1.42e-04, 6.93e-03, 1.16e-02, 1.54e-02) [sam.py:static_binary_density]
12:26:38 INFO : Scatter added after 0.162391 sec [sam.py:static_binary_density]
12:26:38 INFO : 	dens aft: (3.56e-18, 6.31e-13, 2.57e-10, 3.25e-04, 6.99e-03, 1.16e-02, 1.41e-02) [sam.py:static_binary_density]
12:26:38 INFO : 	mass: 2.38e-02 ==> 2.37e-02 || change = -6.2202e-03 [sam.py:static_binary_density]
12:26:38 INFO : zeroing out 6.25e+03/1.56e+04 = 4.00e-01 systems stalled from GMT [sam.py:static_binary_density]
12:26:38 INFO : fraction of coalesced binaries: 4.04e+04/1.88e+05 = 2.16e-01 [sam.py:dynamic_binary_number]
12:26:38 WARNING : WARNING: _stalled_ binaries are not being ac

# Rosado et al 2015

## For GWB
PDF in the absence of a GWB
$$ p_0(S) = \frac{1}{\sqrt{2\pi\sigma_0^2} }e^{-\frac{(S-\mu_0)^2}{2\sigma_0^2}} $$
PDF if GWB is present in the data
 $$ p_1(S) = \frac{1}{\sqrt{2\pi\sigma_1^2} }e^{-\frac{(S-\mu_1)^2}{2\sigma_1^2}} $$

cross correlation
$$ S = \int_{-T/2}^{T/2} dt \int_{-T/2}^{T/2} dt' s_i(t) s_j(t') Q(t,t')$$
$T$ = observing time, $s_i(t)$ and $s_j(t)$ are different pulsar data, $Q(t,t')$ = filter function, chosen to maximize DP for a fixed FAP $=\alpha_0=0.001$ (Neyman-Pearson criterion)


## For single sources
PDF of $\mathcal{F}_e$ - statistic in absence of signal
$$ p_0(\mathcal{F}_e) = \mathcal{F}_e e^{-\mathcal{F}_e}$$
PDF of $\mathcal{F}_e$ - statistic if signal is present
$$ p_1(\mathcal{F}_e, \rho) = \frac{(2\mathcal{F}_e)^{1/2}}{\rho} I_1 (\rho \sqrt{2 \mathcal{F}_e}) e^{-\mathcal{F}_e - 1/2\rho^2} $$
where $I_1(x)$ = the modified Bessel function of the first kind of order 1 and
 $\rho$ = the optimal $S/N_S = \big[ \sum_{i=1}^M S/N_i^2 \big]^{1/2}$

## Detection probability
for single sources: $ \gamma_S(t) = \int_0^t p_s(t') dt'$ \
for background: $ \gamma_B(t) = \int_0^t p_B(t') dt'$

# Variables
* $\alpha$ = false alarm probability
* $\gamma$ = detection probability
* $S_T$ = threshold signal

Background Detection Probability
$$ \gamma_{bg} = \frac{1}{2} \mathrm{erfc} \big[ \frac{\sqrt{2} \sigma_0 \mathrm{erfc}^{-1}(2\alpha_0) - \mu_1}{\sqrt{2} \sigma_1}\big]

In [16]:
ALPHA0 = 0.001 # false alarm probability (FAP)
GAMMA0 = 0.95 # detection probability

def bg_detection_probability(sigma_0, sigma_1, mu_1, alpha_0):
    """ Calculate the background detection probability, gamma_bg (Rosado+2015 Eq. 15).

    Parameters
    ----------
    sigma_0 : scalar
        Standard deviation of stochastic noise processes.
    sigma_1 : scalar
        Standard deviation of GWB PDF.
    mu_1 : scalar
        Mean of GWB PDF.
    alpha_0 : scalar
        False alarm probability max.

    Returns
    -------
    gamma_bg : scalar
        Background detection probability.

    """
    temp = ((np.sqrt(2) * sigma_0 * sp.erfinv(2*alpha_0) - mu_1)
            /(np.sqrt(2) * sigma_1))
    gamma_bg = .5 * sp.erf(temp)
    return gamma_bg

A-statistic: 
$$ S/N_A = \mu_1/\sigma_0$$
B-statistic:
$$ S/N_B = \mu_1 / \sigma_1 $$

In [17]:
def SNR_A(mu_1, sigma_0):
    """ Calculate the SNR for the A-statistic S/N_A 
    
    Parameters
    ----------
    mu_1 : scalar
        Mean of GWB PDF.
    sigma_0 : scalar
        Standard deviation of noise processes.

    Returns
    -------
    SNR_A : scalar
        Signal to noise ratio for the A-statistic.
    """
    return mu_1/sigma_0

def SNR_B(mu_1, sigma_1):
    """ Calculate the SNR for the B-statistic S/N_B.

    Parameters
    ----------
    mu_1 : scalar
        Mean of GWB PDF.
    sigma_1 : scalar
        Standard deviation of the GWB PDf.

    Returns
    -------
    SNR_B : scalar
        Signal to noise ratio for the B-statistic.
    """
    return mu_1/sigma_1

Threshold signal to noise to have FAP $\alpha <\alpha_0$ and detection probability $\gamma > \gamma_0$

$$ \mathrm{S/N^T_A} = \sqrt{2} \big[ \mathrm{erfc}^{-1}(2\alpha_0) - \frac{\sigma_1}{\sigma_0}\mathrm{erfc}^{-1}(2 \gamma_0) \big] $$

$$\mathrm{S/N^T_B} = \sqrt{2} \big[ \frac{\sigma_0}{\sigma_1} \mathrm{erfc}^{-1}(2\alpha_0) - \mathrm{erfc}^{-1}(2 \gamma_0) \big] $$

$$ \mathrm{S/N^T} \approx \sqrt{2} \big[ \mathrm{erfc}^{-1}(2\alpha_0) - \mathrm{erfc}^{-1}(2 \gamma_0) \big] $$

In [18]:
def SNR_A_thresh(sigma_0, sigma_1, alpha_0, gamma_0):
    """ Calculate the threshold SNR for the A-statistic S/N^T_A 
    to have a FAP < alpha_0 and DP > gamma_0.
    
    Parameters
    ----------
    sigma_0 : scalar
        Standard deviation of noise processes.
    sigma_1 : scalar
        Standard deviation of the GWB
    alpha_0 : scalar
        False alarm probability max.
    gamma_0 : scalar
        Detection probability min.

    Returns
    -------
    SNT_A : scalar
        Signal to noise ratio for the A-statistic.
    """
    SNT_A = np.sqrt(2) * (sp.erfinv(2*alpha_0) 
                          - sigma_1/sigma_0 * sp.erfinv(2*gamma_0))
    return SNT_A

def SNR_B(sigma_0, sigma_1, alpha_0, gamma_0):
    """ Calculate the threshold SNR for the B-statistic S/N^T_B
    to have a FAP < alpha_0 and DP > gamma_0.
    
    Parameters
    ----------
    sigma_0 : scalar
        Standard deviation of noise processes.
    sigma_1 : scalar
        Standard deviation of the GWB
    alpha_0 : scalar
        False alarm probability max.
    gamma_0 : scalar
        Detection probability min.

    Returns
    -------
    SNT_B : scalar
        Signal to noise ratio for the A-statistic.
    """
    SNT_B = np.sqrt(2) * (sigma_0/sigma_1 * sp.erfinv(2*alpha_0) 
                          - sp.erfinv(2*gamma_0))
    return SNT_B

def SNR_approx_thresh(alpha_0, gamma_0):
    """ Calculate the approximate threshold SNR S/N^T for FAP < alpha_0 and DP > gamma_0/
    This approximates sigma_0 ~ sigma_1
    
    Parameters
    ----------
    alpha_0 : scalar
        False alarm probability maximum.
    gamma_0 : scalar
        Detection probability minimum.

    Returns
    -------
    
    """
    SNT = np.sqrt(2) * (sp.erfinv(2*alpha_0) - sp.erfinv(2*gamma_0))

Overlap Reduction Function
$$ \Gamma_{ij} = \frac{3}{2} \gamma_{ij} \ln (\gamma_{ij}) - \frac{1}{4} \gamma_{ij} + \frac{1}{2} + \frac{1}{2}\delta_{ij} $$
$$ \gamma_{ij} = [1-\cos (\theta_{ij})]/2$$

In [19]:
def gammaij_from_thetaij(theta_ij):
    """ 
    gamma_ij for two pulsars of relative angle theta_ij
    """
    return (1-np.cos(theta_ij))/2

def thetaij_from_thetai_thetaj(theta_i, theta_j):
    """ 
    relative angle between two pulsars with angular positions theta_i and theta_j
    """
    return np.abs(theta_i - theta_j)

def dirac_ij(i,j):
    """ 
    the dirac delta function of i,j
    """
    if(i==j): return 1
    else: return 0

def overlap_from_gammaij_diracij(gamma_ij, dirac_ij):
    """ 
    Gamma_i,j as a function of gamma_i,j and diracdelta_i,j
    """
    return (3/2 * gamma_ij *np.log(gamma_ij)
            - 1/4 * gamma_ij
            + 1/2 + dirac_ij)

def overlap_reduction_function(theta_i, theta_j, i, j):
    """ Calculate the overlap reduction function Gamma_i,j as a function of theta_i, theta_j, i, and j.
    
    Parameters
    ----------
    theta_i : scalar
        Angular position of the ith pulsar.
    theta_j : scalar
        Angular position of the jth pulsar.
    i : int
        index, i of the pulsar
    j : int
        index, j of the pulsar

    Returns
    -------
    Gamma_ij : scalar
        The overlap reduction function of the ith and jth pulsars.
    """
    dirac_ij = dirac_ij(i, j)
    theta_ij = thetaij_from_thetai_thetaj(theta_i, theta_j)
    gamma_ij = gammaij_from_thetaij(theta_ij)
    return overlap_from_gammaij_diracij(gamma_ij, dirac_ij)

$S_h$, the one-sided power spectral density of the GW signal in the timing residuals
$$ S_h = \frac{h_c^2}{12 \pi ^2 f_k^3}$$

$S_{h0}$, the expected one-sided power spectral density of the GW signal
$$ S_{h0} = \frac{\mathcal{A}^2 \mathrm{yr}^{-4/3}}{12\pi^2} f^{-13/3} $$
where $\mathcal{A}$ is the fiducial characteristic strain amplitude such that 
$$h_c = \mathcal{A} [f/\mathrm{yr}^{-1}]^{-2/3}$$
Rosado et al. approximate as
$$ S_{h0} \approx S_h = \frac{h_c^2}{12 \pi ^2 f_k^3}$$

In [20]:
def power_spectral_density(h_c, f_k):
    """ Calculate the one-sided power spectral density P_i of the ith pulsar at the kth frequency.

    Parameters
    ----------
    h_c : scalar
        Characteristic strain of the ith pulsar.
    f_k : scalar
        Frequency.
    
    Returns
    -------
    S_h : scalar
        Power spectral density of the ith pulsar at the kth frequency.
        Equivalently S_h0 using Rosado et al.'s approximation.
    """
    S_h =  h_c**3 / (12 * np.pi**2 * f_k**3)
    return S_h

Noise spectral density $S_i$ 
$$ S_i = 2 \Delta t \sigma_i^2 + S_{h,\mathrm{rest}}$$
where $2 \Delta t \sigma_i ^2$ is the contribution from the pulsar's white noise and $S_{h,\mathrm{rest}}$ is from all the other SBHBs except for the max, in our single source detection.

In [ ]:
def white_noise(delta_t, sigma_i):
    """ Calculate the white noise for a given pulsar 2 /Delta t sigma_i^2
    
    Parameters
    ----------
    delta_t : scalar
        Detection cadence.
    sigma_i : scalar
        Error/stdev/variance? for the ith pulsar.

    Returns
    -------
    P_i : scalar
        Noise spectral density for the ith pulsar, for bg detection.
        For single source detections, the noise spectral density S_i must also 
        include red noise from all but the loudest single sources, S_h,rest.

    """
    P_i = 2 * delta_t * sigma_i**2

mu_1, sigma_0, sigma_1 when we maximize the A statistic $\mu/\sigma_0 = \langle X \rangle / \sqrt{\mathrm{var}(X)_0}$

$$ \mu_1 = 2 \sum_k \sum_{ij} \frac{\Gamma_{ij}^2 S_h S_{h0}}{P_i P_j}$$

### B-statistic

$$\mu_1 = 1\sum_f \sum_{ij} \frac{\Gamma_{ij}^2 S_h S_{h0}}{[P_i + S_{h0}] [P_j + S_{h0}] + \Gamma_{ij}^2 S_{h0}^2} $$

$$ \sigma_0^2 = 2\sum_f \sum_{ij} \frac{\Gamma_{ij}^2 S_{h0}^2 P_i P_j  }{\big[ [P_i + S_{h0}] [P_j +S_{h0}] + \Gamma_{ij}^2 S_{h0}^2  \big]^2  } $$

$$ \sigma_1^2 = 2 \sum_f \sum_{ij} \frac{\Gamma_{ij}^2 S_{h0}^2 \big[ [P_i + S_h] [P_j + S_h] + \Gamma_{ij}^2 S_h^2   \big]  }{\big[[P_i + S_{h0}][P_j + S_{h0}] + \Gamma_{ij}^2 S_{h0}^2  \big]^2  } $$


* $S_h(f_k)$ := actual value of the spectral density in the background
$$ S_h = \frac{h_c^2}{12 \pi ^2 f_k^3}$$
* $S_{h0}(f_k)$ := value of the spectral density used to construct the statistic
$$ S_{h0} \approx S_h$$
* $P_i$ = $S_i$ := noise spectral density
$$ S_i = 2 \Delta t \sigma_i^2 + S_{h,\mathrm{rest}}$$
* $2 \Delta t \sigma_i^2$ := the contribution from the pulsar's white noise
* $S_{h,\mathrm{rest}}$ := an additional red noise term produced by all other SBHBs at the same frequency bin
$$ S_{h,\mathrm{rest}} = \frac{h_{c,\mathrm{rest}}^2}{f} \frac{1}{12 \pi^2 f^2}$$


In [21]:
def spectral_density(hc_bg, freqs):
    """ Calculate the spectral density S_h(f_k) ~ S_h0(f_k) at the kth frequency

    Parameters
    ----------
    hc_bg : (F,) 1D array of scalars
        Characteristic strain of the background at each frequency. 
    freqs : (F,) 1Darray of scalars
        Frequency bin centers corresponding to each strain.

    Returns
    -------
    S_h : (F,) 1Darray of scalars
        Actual (S_h) or construction (~S_h0) value of the background spectral density. 
    """

    return hc_bg**2 / (12 * np.pi**2 * freqs**3)


In [36]:
def mean1_Bstatistic_loops(noise_i, overlap_ij, Sh_bg, Sh0_bg):
    """ Calculate mu_1 for the background, by summing over all pulsars and frequencies.
    Assuming the B statistic, which maximizes S/N_B = mu_1/sigma_1
    
    Parameters
    ----------
    noise_i : (P,) 1darray of scalars
        Noise spectral density of each pulsar.
    overlap_ij : (P,P,) 2Darray of scalars
        Overlap reduction function.
    Sh_bg : (F,) 1Darray of scalars
        Spectral density in the background.
    Sh0_bg : (F,) 1Darray of scalars
        Value of spectral density used to construct the statistic.

    Returns
    -------
    mu_1B : 
        Expected value for the B statistic
    """
    mu_1B = 0
    for ii in range(len(noise_i)):
        P_i = noise_i[ii]
        for jj in range(len(noise_i)): # should ii and jj repeat, or just do for jj < ii?
            P_j = noise_i[jj]
            Gamma_ij = overlap_ij[ii,jj]
            for kk in range(len(Sh_bg)):
                Sh_k = Sh_bg[kk]
                Sh0_k = Sh0_bg[kk]
                mu_1B += ((Gamma_ij**2 * Sh_k * Sh0_k)
                          /((P_i+Sh0_k) * (P_j+Sh0_k) 
                            + Gamma_ij**2 * Sh0_k**2))
    mu_1B *= 2
    return mu_1B

def mean1_Bstatistic_ndars(noise_i, overlap_ij, Sh_bg, Sh0_bg):
    """ Calculate mu_1 for the background, by summing over all pulsars and frequencies.
    Assuming the B statistic, which maximizes S/N_B = mu_1/sigma_1
    
    Parameters
    ----------
    noise_i : (P,) 1darray of scalars
        Noise spectral density of each pulsar.
    overlap_ij : (P,P,) 2Darray of scalars
        Overlap reduction function.
    Sh_bg : (F,) 1Darray of scalars
        Spectral density in the background.
    Sh0_bg : (F,) 1Darray of scalars
        Value of spectral density used to construct the statistic.

    Returns
    -------
    mu_1B : 
        Expected value for the B statistic
    """

    # to get sum term in shape (P,P,F) for ii,jj,kk we want:
    # Gamma_ij in shape (P,P,1)
    # Sh0 and Sh in shape (1,1,F)
    # P_i in shape (P,1,1)
    # P_j in shape (1,P,1)

    numer = (overlap_ij[:,:,np.newaxis] **2 
            * Sh_bg[np.newaxis, np.newaxis, :]
            * Sh0_bg[np.newaxis, np.newaxis, :])
    denom = ((noise_i[:, np.newaxis, np.newaxis] + Sh0_bg[np.newaxis,np.newaxis,:])
               * (noise_i[np.newaxis, :, np.newaxis] + Sh0_bg[np.newaxis,np.newaxis,:])
               + overlap_ij[:,:,np.newaxis]**2 * Sh0_bg[np.newaxis, np.newaxis, :]**2)
    mu_1B = 2*np.sum(numer/denom)
    return mu_1B


In [ ]:
# better to do with ndars, construct Gamma_ij as 2darray
THETAS = np.linspace(0, 2*np.pi, 4) # (P,) 1Darray of scalars, angular sky position of each pulsar
SIGMAS = np.linspace(1,1,4) # (P,) 1Darray of scalars, sigma_i of each pulsar
print(THETAS, SIGMAS)

num = len(THETAS) # number of pulsars, P
overlap_ij = np.array((num, num)) # (P,P) 2Darray of scalars, Overlap reduction function between all puolsar
for ii in range(num):
    for jj in range(num):
        overlap_ij[ii,jj] = overlap_reduction_function(THETAS[ii], THETAS[jj], ii, jj)
noise_i = white_noise(cad, SIGMAS)
Sh_bg = spectral_density(hc_bg, fobs) # spectral density of bg
Sh0_bg = Sh_bg # approximation used in Rosado et al. 2015
mu_1B = mean1_Bstatistic_loops(noise_i, overlap_ij, Sh_bg, Sh0_bg)
